# Model Deployment with Gradio and Hugging Face

- We trained our model and say we are happy about its performance. We may want to introduce it community so that everyone can use it, preferably via their phones. This is called *deployment*. 

- Core idea is that users will take or upload their sneaker pictures to a server and we will pass the image through our model and present the results back to the user in a user-friendly manner. One of the most popular tools to achive this is **gradio**. 

- Once we have the best Pytorch model, all we need is a function,*get_class_probs*, that can accept the image and return the predictions,

In [1]:
!pip install gradio timm -q
import gradio as gr
import torch
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import timm

In [2]:
idx_to_class = {0: 'adidas', 1: 'converse', 2: 'new-balance', 3: 'nike', 4: 'reebok', 5: 'vans'}
num_classes = len(idx_to_class)

mean = [0.485, 0.456, 0.406]
std =  [0.229, 0.224, 0.225]
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                     transforms.Normalize(mean,std)
                                     ])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def get_model(model_name = 'efficientnet_b0',freeze = False):
    model = timm.create_model(model_name = model_name,pretrained=True)
    if freeze:
        for parameter in model.parameters():
            parameter.requires_grad = False
    
    in_features = model.classifier.in_features # 1792
    
    model.classifier = nn.Sequential(
                          nn.Linear(in_features, 100), 
                          nn.BatchNorm1d(num_features=100),
                          nn.ReLU(),
                          nn.Dropout(),
                          nn.Linear(100, num_classes),
                                    )
    
    return model

In [4]:
def load_model(model, model_path):
    checkpoint = torch.load(model_path,map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.best_scores = checkpoint['best_stats']
    return model

In [5]:
model = load_model(get_model(),"snicker_model.pth")

In [6]:
def get_class_probs(img):
    with torch.no_grad():
        model.eval()
        model.to(device)
        img = test_transforms(img)
        img = img.unsqueeze(0).to(device)
        output = model(img)
        # remember softmax
        probs = F.softmax(output,dim=1)
        probs, indices = probs.topk(k=num_classes)
        probs = probs[0].tolist()
        indices = indices[0].tolist()
        classes = [idx_to_class[index] for index in indices]
        confidences = {classes[i]: round(probs[i],3) for i in range(num_classes)}  

    return confidences

In [7]:
examples = ["samples/a.jpeg","samples/c.jpeg","samples/r.jpeg"]
gr.Interface(fn=get_class_probs, 
             inputs=gr.Image(type="pil",source="upload"),
             outputs=gr.Label(num_top_classes=6),
             examples=examples).launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f4623906-a7cb-41b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


- As you see from the message above, the link we create expires in 72 hours. If you need a permenant deployment, you can use [HuggingFace](https://huggingface.co/). If you want to be an AI practioner, this is one of the must spaces you should discover. 

- All we need is to create an account and upload eveything in this folder. "app.py" is just a naive Python file of this jupyter file. Huggingface system will compile it and create your app automatically. I already did it. Here is the permanent link


https://huggingface.co/spaces/erdi28/sneaker-app